# GeoST Accessors
This page briefly goes into more detail about the header and data accessors available for
Geopandas GeoDataFrames and Pandas DataFrames as shown on the [previous page](./data_structures.ipynb#geost_accessors). For header GeoDataFrames the [`.gsthd`](../api_reference/header_accessors.rst)
accessor is available and for data DataFrames the [`.gstda`](../api_reference/data_accessors.rst)
is available. For these accessors to work, the GeoDataFrame or DataFrame must meet some criteria
which will be explained below. Additionally, we will provide some details about how they
work with several data sources for a better understanding on their use and functionality.

We will use the GeoST borehole and CPT sample data for this exaplanation.


In [3]:
import geost

geost.config.validation.VERBOSE = False  # Ignore validation warning in CPTs

# Load sample data
cores = geost.data.boreholes_usp()
cpts = geost.data.cpts_usp()

# Separate header and data tables
cores_header, cores_data = cores.header, cores.data
cpts_header, cpts_data = cpts.header, cpts.data


NOTE: Header has been reset to align with data because AUTO_ALIGN is enabled in the GeoST configuration.


C:\src\geost\geost\base.py:307: AlignmentWarning: Header covers more/other objects than present in the data table. consider running the method 'reset_header' to update the header.
  warnings.warn(


## Header `.gsthd` accessor
The [`.gsthd`](../api_reference/header_accessors.rst) only works on Geopandas GeoDataFrames
which is related to different types of subsurface data. Typically available types of subsurface
data comprise point-like data such as boreholes, cpts, well logs and line-like data such as
seismics, GPR, EM. This means that the location of different types of surveys in some cases
are represented by a point and others by a line. In the design, GeoST aims to be as consistent
as possible with creating methods (e.g. `select_within_bbox`, `select_with_points`) but under
the hood, some methods may need to work slightly different based on the type of geometry.

The `.gsthd` accessor automatically resolves the above problem and figures out the correct method
based on the datatype in the "geometry" column of a GeoDataFrame. This is done by determining
the geometry type of the first row in the GeoDataFrame and selecting the required backend:
for instance a [`PointHeader`](../api_reference/header_accessors.rst#pointheader) in case of
point data and [`LineHeader`](../api_reference/data_accessors.rst#lineheader) in case of line
data. To illustrate this, we will stepwise show how this works the loaded `cores_header`.

Let's first check the geometry type in the header table and see what happens when simply
use the `.gsthd` attribute.

In [ ]:
print(cores_header.geom_type.head())  # Only print the first 5 rows
cores_header.gsthd

0    Point
1    Point
2    Point
3    Point
4    Point
dtype: object


As you can see, the geometry type is "Point" and calling the `.gsthd` attribute prints a
`geost.accessors.accessor.Header` instance. This is a generic class where the correct backend
is chosen based on the geometry type. Since the geometry type is "Point" we expect that the
chosen backend is a `PointHeader`. We can check this by checking the resulting backend.

In [8]:
cores_header.gsthd._backend

We see that the backend indeed is a `PointHeader` instance. This makes sure that each called
method when using the accessor is dispatched to the PointHeader instance.

## Data `.gstda` accessor